In [13]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
%matplotlib inline

In [2]:
path = '/home/sadat/Documents/DSE/Data/yogurt.h5'

In [3]:
store = pd.HDFStore(path)
nrows = store.get_storer('groc').nrows
r = np.random.randint(0,nrows,size=10000)
store_df=pd.read_hdf(path,'groc',where=pd.Index(r))
store_df.head()

,IRI_KEY,WEEK,SY,GE,VEND,ITEM,UNITS,DOLLARS,F,D,PR,file,year_int
7506014,253671,1139,0,1,70470,116,46,18.40,B,0,1,yogurt_groc_1114_1165,1
10052322,223759,1621,0,1,52159,1330,6,20.94,NONE,0,0,yogurt_groc_1583_1634,10
9600448,1109318,1423,0,1,70640,50203,7,4.83,NONE,0,0,yogurt_groc_1374_1426,6
4495440,242405,1115,0,1,70470,123,53,26.50,B,0,1,yogurt_groc_1114_1165,1
831946,200635,1323,0,3,36632,620,20,8.00,B,0,1,yogurt_groc_1322_1373,5


Taking values before 2007 and changing D so that it is 1 if display is used else 0

In [4]:
#Sales before 2007 were stable
store_pre07=store_df[store_df['year_int']<7]

#We dont care about the size of the display and hence setting D=2 to 1.
store_pre07[store_pre07['D']==2] = 1

store_pre07 = store_pre07[['UNITS','DOLLARS','D','PR','SY','GE','VEND','ITEM']]

store_pre07.head()

/home/sadat/anaconda3/envs/py2k/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sadat/anaconda3/envs/py2k/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,UNITS,DOLLARS,D,PR,SY,GE,VEND,ITEM
7506014,46,18.40,0,1,0,1,70470,116
9600448,7,4.83,0,0,0,1,70640,50203
4495440,53,26.50,0,1,0,1,70470,123
831946,20,8.00,0,1,0,3,36632,620
7379501,8,23.44,0,0,0,1,75270,194


Creating dummies in Products data

In [5]:
prod_path='/home/sadat/Desktop/IRI Yogurt/parsed stub files/prod_yogurt.xls'
df = pd.read_excel(prod_path)

dummies_L1=pd.get_dummies(df['L1'],prefix='L1')
dummies_PROD=pd.get_dummies(df['PRODUCT TYPE'],prefix='PROD')
dummies_FLAV=pd.get_dummies(df['FLAVOR/SCENT'],prefix='FLAV')
dummies_FAT=pd.get_dummies(df['FAT CONTENT'],prefix='FAT')
dummies_CAL=pd.get_dummies(df['CALORIE LEVEL'],prefix='CAL')
dummies_STYLE=pd.get_dummies(df['STYLE'],prefix='STYLE')
dummies_TYPE=pd.get_dummies(df['TYPE OF YOGURT'],prefix='TYPE')
dummies_PACK=pd.get_dummies(df['PACKAGE'],prefix='PACK')
prod = pd.concat([
       dummies_L1, dummies_PROD, dummies_PACK,
        dummies_FLAV, dummies_FAT, dummies_CAL,
        dummies_STYLE, dummies_TYPE, df
    ],axis=1)
prod = prod.drop(labels=['L1','L2','L3','L4','L5','L9','VOL_EQ','PRODUCT TYPE','FLAVOR/SCENT',
          'FAT CONTENT', 'CALORIE LEVEL', 'STYLE', 'TYPE OF YOGURT','VOL_EQ', 'PACKAGE','Level',
          '*STUBSPEC 1284RC                                                         00004  ','UPC'
                        ], axis=1)
prod.head()

,L1_CATEGORY - YOGURT,PROD_ACIDOPHILUS YOGURT,PROD_BUFFALO MILK YOGURT,PROD_CULTURED DAIRY DRINK,PROD_CULTURED SOY DRINK,PROD_CULTURED SOY SMOOTHI,PROD_DAIRY BLEND,PROD_DAIRY SNACK,PROD_GOATMILK YOGURT,PROD_GOATSMILK YOGRT DRNK,...,TYPE_REGULAR,TYPE_REGULAR STIR IN,TYPE_SAUCE ON THE BOTTOM,TYPE_STIR IN,TYPE_SYRUP ON THE BOTTOM,TYPE_WHIPPED,SY,GE,VEND,ITEM
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7,2,20742,7011
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,3,70452,24101
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7,3,20742,7013
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,7,2,20742,7012
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0,3,70452,24102


Merge grocery store and products data

In [6]:
prod_store_pre_07=prod.merge(store_pre07, how='inner')
prod_store_pre_07.head()

,L1_CATEGORY - YOGURT,PROD_ACIDOPHILUS YOGURT,PROD_BUFFALO MILK YOGURT,PROD_CULTURED DAIRY DRINK,PROD_CULTURED SOY DRINK,PROD_CULTURED SOY SMOOTHI,PROD_DAIRY BLEND,PROD_DAIRY SNACK,PROD_GOATMILK YOGURT,PROD_GOATSMILK YOGRT DRNK,...,TYPE_SYRUP ON THE BOTTOM,TYPE_WHIPPED,SY,GE,VEND,ITEM,UNITS,DOLLARS,D,PR
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7,1,20742,7004,5,3.45,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,21000,820,2,4.00,0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,21000,820,1,2.39,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,21000,820,6,13.08,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,21000,12585,1,2.79,0,0


Calculate per unit price 

In [7]:
prod_store_pre_07['UNIT_PRICE']=prod_store_pre_07['DOLLARS']/prod_store_pre_07['UNITS']
prod_store_pre_07 = prod_store_pre_07.drop(['SY','GE','VEND','ITEM','DOLLARS'],axis=1)
prod_store_pre_07.head()

,L1_CATEGORY - YOGURT,PROD_ACIDOPHILUS YOGURT,PROD_BUFFALO MILK YOGURT,PROD_CULTURED DAIRY DRINK,PROD_CULTURED SOY DRINK,PROD_CULTURED SOY SMOOTHI,PROD_DAIRY BLEND,PROD_DAIRY SNACK,PROD_GOATMILK YOGURT,PROD_GOATSMILK YOGRT DRNK,...,TYPE_REGULAR,TYPE_REGULAR STIR IN,TYPE_SAUCE ON THE BOTTOM,TYPE_STIR IN,TYPE_SYRUP ON THE BOTTOM,TYPE_WHIPPED,UNITS,D,PR,UNIT_PRICE
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0,0.69
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,2.00
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,2.39
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,0,0,2.18
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,2.79


Set UNITS and UNIT_PRICE so that they are 1 if greater than median, else 0

In [8]:
prod_store_pre_07['UNIT_PRICE']=prod_store_pre_07['UNIT_PRICE'].apply(lambda x:
                                                                      1 if 
                                                                      x>prod_store_pre_07['UNIT_PRICE'].median() 
                                                                      else 0)

prod_store_pre_07['UNITS']=prod_store_pre_07['UNITS'].apply(lambda x:
                                                                1 if 
                                                                x>prod_store_pre_07['UNITS'].median() 
                                                                else 0)

In [9]:
prod_store_pre_07.head()

,L1_CATEGORY - YOGURT,PROD_ACIDOPHILUS YOGURT,PROD_BUFFALO MILK YOGURT,PROD_CULTURED DAIRY DRINK,PROD_CULTURED SOY DRINK,PROD_CULTURED SOY SMOOTHI,PROD_DAIRY BLEND,PROD_DAIRY SNACK,PROD_GOATMILK YOGURT,PROD_GOATSMILK YOGRT DRNK,...,TYPE_REGULAR,TYPE_REGULAR STIR IN,TYPE_SAUCE ON THE BOTTOM,TYPE_STIR IN,TYPE_SYRUP ON THE BOTTOM,TYPE_WHIPPED,UNITS,D,PR,UNIT_PRICE
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1


Classification

In [10]:
y = prod_store_pre_07['UNITS']
X = prod_store_pre_07.drop('UNITS',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44)

In [14]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.66167290886392005

In [12]:
clf = LinearSVC(C=0.3, penalty='l2')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.68164794007490637